<a href="https://colab.research.google.com/github/dsaha78/mtech/blob/main/02ml/ml_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd

In [12]:
xlsx = pd.ExcelFile('https://raw.githubusercontent.com/dsaha78/mtech/main/02ml/input/Plant_Tree_data.xlsx')
dataset = pd.read_excel(xlsx, 'Sheet1',header=0)
print(dataset.head)

<bound method NDFrame.head of        LeafLength  LeafBreadth  ...  TreeLength  TreeType
0      140.562500    55.683782  ...   74.242225         0
1      102.507812    58.882430  ...  127.393580         0
2      103.015625    39.341649  ...   63.171909         0
3      136.750000    57.178449  ...   53.593661         0
4       88.726562    40.672225  ...  252.567306         0
...           ...          ...  ...         ...       ...
17893  136.429688    59.847421  ...  285.931022         0
17894  122.554688    49.485605  ...    8.297092         0
17895  119.335938    59.935939  ...    4.595173         0
17896  114.507812    53.902400  ...  134.238910         0
17897   57.062500    85.797340  ...    1.429475         0

[17898 rows x 9 columns]>
